In [ ]:
pip install progressbar2

In [ ]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import progressbar

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
label_df = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')
labels = dict(zip(label_df['segment_id'].astype(str), label_df['time_to_eruption']))
#print(labels)


def normalize_df(df):
    return (df-df.min())/(df.max()-df.min())


def read_segment(segment_id, train):
    #print(f'Read {segment_id}')
    if train:
        segment_path = f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{segment_id}.csv'
    else:
        segment_path = f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/{segment_id}.csv'
        
    segment = pd.read_csv(segment_path)
    #print(segment.isnull().any())
    #display(segment)
    segment = segment.fillna(segment.mean())
    #display(segment)
    nan_columns = segment.isnull().any().values
    is_any_col_nan = (sum(nan_columns*1)>0)
    segment = segment.fillna(0)
    
    segment = normalize_df(segment)

    if is_any_col_nan:
        drop=True
    else:
        drop=False
        # drop this datapoint
    # read df
    # fill NaN
    # convert to numpy tensor
    return segment, drop


def create_data(segment_ids, eruption_times, train):
    max_val = 0
    dropped=0
    #X = np.empty((4431, 60001, 10))
    y = eruption_times
    if not os.path.exists("/kaggle/data/segments"):
        os.makedirs("/kaggle/data/segments")
    with progressbar.ProgressBar(max_value=len(segment_ids)) as bar:
        for idx, seg_id in progressbar.progressbar(enumerate(segment_ids)):
            segment, drop = read_segment(seg_id, train)
            if drop:
                dropped+=1

            bar.update(idx)

                
            segment.to_csv(f"/kaggle/data/segments/{str(seg_id)}.csv", index=False)
    print(f"{dropped} data points with missing sensors")
   
    label_df.to_csv(f"/kaggle/working/data/train.csv")
    
create_data(list(labels.keys())[:], label_df['time_to_eruption'], True)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import progressbar
from multiprocessing import Pool
from functools import partial
import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
label_df = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')
labels = dict(zip(label_df['segment_id'].astype(str), label_df['time_to_eruption']))
#print(labels)


def normalize_df(df):
    return (df-df.min())/(df.max()-df.min())


def read_segment(segment_id, train):
    #print(f'Read {segment_id}')
    if train:
        segment_path = f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{segment_id}.csv'
    else:
        segment_path = f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/{segment_id}.csv'
        
    segment = pd.read_csv(segment_path)
    #print(segment.isnull().any())
    #display(segment)
    segment = segment.fillna(segment.mean())
    #display(segment)
    nan_columns = segment.isnull().any().values
    is_any_col_nan = (sum(nan_columns*1)>0)
    segment = segment.fillna(0)
    
    segment = normalize_df(segment).astype('float16')

    if is_any_col_nan:
        drop=True
    else:
        drop=False
        # drop this datapoint
    # read df
    # fill NaN
    # convert to numpy tensor
    write_segment(segment_id, segment, train)
    return segment, drop


def write_segment(seg_id, segment, train):
    segment.to_csv(f"/kaggle/working/data/segments/{str(seg_id)}.csv", index=False)
    #print(f'written: {seg_id}')


def create_data(segment_ids, eruption_times, train):
    max_val = 0
    dropped=0
    #X = np.empty((4431, 60001, 10))
    y = eruption_times
    if not os.path.exists("/kaggle/working/data/segments"):
        os.makedirs("/kaggle/working/data/segments")

    with Pool(4) as p:
        #r = list(tqdm.tqdm(p.imap(_foo, range(30)), total=30))
        result = list(tqdm.tqdm(p.imap(partial(read_segment, train=True), segment_ids), total=len(segment_ids)))
    print(f"{dropped} data points with missing sensors")
   
    label_df.to_csv(f"/kaggle/working/data/train.csv")
    
create_data(list(labels.keys())[:], label_df['time_to_eruption'], True)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/predict-volcanic-eruptions-ingv-oe/train | wc
!ls /kaggle/input/predict-volcanic-eruptions-ingv-oe/test | wc

In [ ]:
!ls /kaggle/working


In [ ]:
df=pd.read_csv('/kaggle/working/data/segments/139656908.csv')
display(df)